In [1]:
import os, re, sys
from pymongo import MongoClient
from pprint import pprint

In [2]:
client = MongoClient()
db = client.tg3

In [3]:
# list all projects and subprojects
project_list = db.projects.find()



In [99]:
# get project P43
p_id = 48
p = db.projects.find_one({"_id": p_id})
p

{u'_id': 48,
 u'dateCreated': datetime.datetime(2012, 9, 27, 20, 8, 12),
 u'dmID': None,
 u'internalIndex': 48,
 u'isActive': True,
 u'lastUpdated': datetime.datetime(2012, 9, 27, 20, 8, 58),
 u'projectBackground': u'',
 u'projectGoals': u'',
 u'projectIRB': None,
 u'projectMethods': u'',
 u'projectResults': u'',
 u'projectTitle': u'Pilot Sing Cell C1 Profiling',
 u'projectType': u'Experiment',
 u'searchId': u'p48',
 u'subprojects': [{u'dateCreated': datetime.datetime(2012, 9, 27, 20, 8, 12),
   u'experimentName': u'Pilot Sing Cell C1 Profiling',
   u'projectType': u'Experiment',
   u'subprojectId': 0}]}

In [48]:
# parse subproject experiment name to extract experiment type
def get_experiment_type(experiment_name):
    e_type_dict = {'microarray': 'microarray',
                   'fluidigm': 'fluidigm',
                   'c1': 'sc_rnaseq',
                   'bulk': 'bulk_rnaseq'}
    
    e_type_match = [ e_type_dict[t] \
                     for t in e_type_dict \
                     if re.search(t, experiment_name.lower()) ]
    if len(e_type_match):
        e_type = e_type_match[0]
    else:
        e_type = 'unknown'
    return e_type

# parse subproject experiment name to extract month (and year, if available)
def get_experiment_date(experiment_name):
    m_dict = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
              'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
    
    e_month_match = [ m_dict[m] for m in m_dict if re.search(m, experiment_name.lower()) ]
    if len(e_month_match):
        e_month = e_month_match[0]
    else:
        e_month = 'unknown'
        
    e_year_match = re.search('[0-9]{4}', experiment_name)
    if e_year_match is not None:
        e_year = int(e_year_match.group())
    else:
        e_year = 'unknown'
    
    return(e_month, e_year)

In [57]:
s = p.get('subprojects')[15]
print s.get('experimentName')
print get_experiment_type(s.get('experimentName'))
print get_experiment_date(s.get('experimentName'))

renamed bulk from Nov25_2014
bulk_rnaseq
(11, 2014)


In [74]:
# pull out subproject info
subproject_dict = { s.get('subprojectId'): \
                    {'experimentType': get_experiment_type(s.get('experimentName')),
                     'experimentDate': 
                     {'month': get_experiment_date(s.get('experimentName'))[0],
                      'year': get_experiment_date(s.get('experimentName'))[1]}} \
                    for s in p.get('subprojects') }

In [75]:
subproject_dict

{0: {'experimentDate': {'month': 'unknown', 'year': 'unknown'},
  'experimentType': 'unknown'},
 1: {'experimentDate': {'month': 6, 'year': 2012},
  'experimentType': 'microarray'},
 2: {'experimentDate': {'month': 11, 'year': 2012},
  'experimentType': 'fluidigm'},
 3: {'experimentDate': {'month': 3, 'year': 2013},
  'experimentType': 'microarray'},
 4: {'experimentDate': {'month': 5, 'year': 2013},
  'experimentType': 'fluidigm'},
 5: {'experimentDate': {'month': 4, 'year': 2014},
  'experimentType': 'sc_rnaseq'},
 6: {'experimentDate': {'month': 8, 'year': 2014},
  'experimentType': 'sc_rnaseq'},
 7: {'experimentDate': {'month': 11, 'year': 2014},
  'experimentType': 'fluidigm'},
 8: {'experimentDate': {'month': 11, 'year': 2014},
  'experimentType': 'sc_rnaseq'},
 9: {'experimentDate': {'month': 11, 'year': 2014},
  'experimentType': 'sc_rnaseq'},
 10: {'experimentDate': {'month': 2, 'year': 2015},
  'experimentType': 'unknown'},
 11: {'experimentDate': {'month': 2, 'year': 2015},


In [79]:
# guess experiment year based on year of preceding subproject
def infer_experiment_year(subproject_dict):
    for s in subproject_dict:
        if s is not 0 and subproject_dict[s]['experimentDate']['year'] is 'unknown':
            e_year_guess = subproject_dict[s - 1]['experimentDate']['year']
            subproject_dict[s]['experimentDate']['year'] = e_year_guess
    return subproject_dict

In [127]:
def get_fc_id(fc_str):
    fc_str = re.sub('EXTERNAL_(A|B|D)', 'EXTERNAL_', fc_str)
    fc_re = re.compile('((?<=(EXTERNAL_))|(?<=(_(A|B|D))))([A-Z]|[0-9])*XX')
    fc_match = fc_re.search(fc_str)
    if fc_match is not None:
        fc_id = fc_match.group()
    else:
        fc_id = ''
        print fc_str

    return fc_id

# read and extract info from library list file
def read_lib_list(lib_list_file):
    with open(lib_list_file) as f:
        lib_list_lines = f.readlines()

    lib_fastq_dict = {}
    for l in lib_list_lines[1:]:
        l_parts = l.strip().split('\t')
        
        lib_id_match = re.search('lib[1-9]+[0-9]*', l_parts[0])
        if lib_id_match is not None:
            lib_fastq_dict[lib_id_match.group()] = \
                {'flowcellRun': 
                     {'year': 2000 + int(l_parts[2][0:2]),
                      'flowcellId': get_fc_id(l_parts[2])}, 
                 'projectName': l_parts[3],
                 'fastqPath': l_parts[-1]}
        else:
            print l_parts[0]
    
    return lib_fastq_dict

In [83]:
lib_list_file = "../data/lib_list_master.txt"
lib_list_lines = read_lib_list(lib_list_file)

In [128]:
read_lib_list(lib_list_file)

lib001
lib0010
lib00100
lib00101
lib00102
lib00103
lib00104
lib00105
lib00106
lib00107
lib00108
lib00109
lib0011
lib00110
lib00111
lib00112
lib00113
lib0012
lib0013
lib0014
lib0015
lib0016
lib0017
lib0018
lib0019
lib002
lib0020
lib0021
lib0022
lib0023
lib0024
lib0025
lib0026
lib0027
lib0028
lib0029
lib003
lib0030
lib0031
lib0032
lib0033
lib0034
lib0035
lib0036
lib0037
lib0038
lib0039
lib004
lib0040
lib0041
lib0042
lib0043
lib0044
lib0045
lib0046
lib0047
lib0048
lib0049
lib005
lib0050
lib0051
lib0052
lib0053
lib0054
lib0055
lib0056
lib0057
lib0058
lib0059
lib006
lib0060
lib0061
lib0062
lib0063
lib0064
lib0065
lib0066
lib0067
lib0068
lib0069
lib007
lib0070
lib0071
lib0072
lib0073
lib0074
lib0075
lib0076
lib0077
lib0078
lib0079
lib008
lib0080
lib0081
lib0082
lib0083
lib0084
lib0085
lib0086
lib0087
lib0088
lib0089
lib009
lib0090
lib0091
lib0092
lib0093
lib0094
lib0095
lib0096
lib0097
lib0098
lib0099
rename_lib_folders.sh
sample_lib_key.csv


{'lib1000': {'fastqPath': '/net/srvnas01/GENOMICS/Illumina/130725_H135_0278_AC26AHACXX/Unaligned/Project_P47/Sample_lib1000',
  'flowcellRun': {'flowcellId': 'C26AHACXX', 'year': 2013},
  'projectName': 'Project_P47'},
 'lib10000': {'fastqPath': '/mnt/genomics/Illumina/151216_D00565_0100_AC81A1ANXX/Unaligned/P43-29-27330309/lib10000-31653654',
  'flowcellRun': {'flowcellId': 'C81A1ANXX', 'year': 2015},
  'projectName': 'P43-29-27330309'},
 'lib10001': {'fastqPath': '/mnt/genomics/Illumina/151216_D00565_0100_AC81A1ANXX/Unaligned/P43-29-27330309/lib10001-31660659',
  'flowcellRun': {'flowcellId': 'C81A1ANXX', 'year': 2015},
  'projectName': 'P43-29-27330309'},
 'lib10002': {'fastqPath': '/mnt/genomics/Illumina/151216_D00565_0100_AC81A1ANXX/Unaligned/P43-29-27330309/lib10002-31657668',
  'flowcellRun': {'flowcellId': 'C81A1ANXX', 'year': 2015},
  'projectName': 'P43-29-27330309'},
 'lib10003': {'fastqPath': '/mnt/genomics/Illumina/151216_D00565_0100_AC81A1ANXX/Unaligned/P43-29-27330309/li

In [60]:
{ s.get('subprojectId'): s.get('experimentName') for s in p.get('subprojects')}

{0: u'Eric Wambre',
 1: u'Jun2012 Microarray',
 2: u'Nov2012 Fluidigm',
 3: u'Mar2013 Microarray',
 4: u'May2013 Fluidigm',
 5: u'Apr2014 (C1)Nextera Library Sequencing',
 6: u'Aug2014 (C1)',
 7: u'Nov2014 Fluidigm',
 8: u'C1 Nov2014',
 9: u'C1 Nov_25_2014',
 10: u'ITN Grass allergy Feb2015',
 11: u'Feb27-2015 Cells Direct Fluidigm',
 12: u'Mar19-May7 2015 2015 Bulk',
 13: u'Mar25 direct C1 capture',
 14: u'Mar25 Cells Direct Fluidigm',
 15: u'renamed bulk from Nov25_2014',
 16: u'ITN-Cells Direct-Grass allergy April2015',
 17: u'Bulk 03June2015',
 18: u'10Jun2015 C1 directsort',
 19: u'7July2015 bulk',
 20: u'20Aug2015 C1',
 21: u'25Aug2015 C1',
 22: u'26Aug2015 C1',
 23: u'27Aug2015 C1',
 24: u'27Aug2015 Bulk',
 25: u'16Sep 2015 Bulk',
 26: u'11Nov2015 Cell Direct Fluidigm',
 27: u'C1- PA#229',
 28: u'C1 HC26 19Nov2015',
 29: u'C1 Nov. 24, 2015',
 30: u'Bulk Nov. 25, 2015',
 31: u'02Dec2015 C1',
 32: u'03Dec2015 C1',
 33: u'03Dec2015 Bulk',
 34: u'09Dec2015 C1',
 35: u'C1 10Dec2015',

In [5]:
def listdir_re(path, regex):
    file_list = [ os.path.join(dp, f) \
                  for dp,dn,fn in os.walk(path) \
                  for f in fn if re.search(regex, f) ]
    return file_list

In [6]:
project_dir = "/Volumes/DFS_Chaussabel_LabShare/Projects/"
lib_prep_files = listdir_re(project_dir, '(L|l)ibrary.*xls')

In [113]:
date_str = 'Mar2014'

In [114]:
re.search('[0-9]{4}', date_str).group()

'2014'

In [96]:
lib_id = "lib10"
lib = db.samples.find_one({"_id": lib_id})
cdna = db.samples.find_one({"_id": lib.get("parentId")})
sample = db.samples.find_one({"_id": cdna.get("parentId")})

In [97]:
lib

{u'_id': u'lib10',
 u'barcode': None,
 u'concentration': 30.799999237060547,
 u'dateCreated': datetime.datetime(2011, 9, 27, 21, 33, 1),
 u'internalIndex': 10,
 u'lastUpdated': datetime.datetime(2011, 11, 3, 19, 8, 58),
 u'libraryId': u'lib10',
 u'libraryIndex': 7,
 u'notes': None,
 u'nqConcentration': None,
 u'nqMolarity': None,
 u'parentId': u'grRNA982',
 u'peak': None,
 u'processingDate': None,
 u'processing_date': None,
 u'projectId': 28,
 u'projectName': u'RRMS-CIS-NMO',
 u'purity': 0.0,
 u'qpcrConcentration': None,
 u'removed': False,
 u'researcher': None,
 u'sampleId': u'S3366',
 u'size': 267,
 u'storage': None,
 u'type': u'library'}

In [ ]:
p48_lids = db.samples.find()

In [110]:
x = '1512'

In [112]:
x[0:2]

'15'